<a href="https://colab.research.google.com/github/sickopickle/NoTeX/blob/main/Project_(fixed_fraction_bar_and_removed_complex_functions_and_spatial_relations).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!sud                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              `````````````````````````````````````````                                                                     ,,,o add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [35]:
from bs4 import BeautifulSoup
import tensorflow as tf
import numpy as np

In [36]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

/content
mkdir: cannot create directory ‘drive’: File exists
/content/drive
mkdir: cannot create directory ‘MyDrive’: File exists
/content
/
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [37]:
data_file_path="C:/Users/aiden/Downloads/ProjectData/LGINKML/"
lg_file_path=data_file_path+"LGs/"
inkml_file_path=data_file_path+"INKMLs/"

In [38]:
def getSoup(file):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  return soup


In [39]:
def arrayToPoints (arr) :
  points = []
  for i in arr:
    points.append(point(float(i[0]), float(i[1])))
  return(points)
  
def PointsToArray (points) :
  arr=[]
  for point in points:
    arr.append([point.x,point.y])
  return arr

In [40]:
import numpy as np
import math
EPSILON = 1e-12
MACHINE_EPSILON = 1.12e-16


class point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
def isMachineZero (val) : 
  return (val >= -MACHINE_EPSILON and val <= MACHINE_EPSILON)
def hypot (x, y) : 
  return math.sqrt(x * x + y * y)
def pointLength (p) : 
  return hypot(p.x, p.y)
def pointNegate (p) : 
  return point(-p.x, -p.y)
def pointAdd (p1, p2) : 
  return point(p1.x + p2.x, p1.y + p2.y)
def pointSubtract (p1, p2) : 
  return point(p1.x - p2.x, p1.y - p2.y)
def pointMultiplyScalar (p, n) : 
  return point(p.x * n, p.y * n)
def pointDot (p1, p2) : 
  return p1.x * p2.x + p1.y * p2.y
def pointDistance (p1, p2) : 
  return hypot(p1.x - p2.x, p1.y - p2.y)
def pointNormalize (p, length = 1) : 
  return pointMultiplyScalar(p, (length / pointLength(p)) if pointLength(p) else 0) #POSSIBLE FLAW
class createSegment: 
  def __init__(self, p, i, o):
    self.p=p
    self.i=i
    self.o=o #POSSIBLE FLAW (CHANGED)

def chordLengthParameterize (points, first, last)  :
    u = [0]
    for i in range(first+1, last+1):
      u.append(u[i - first - 1] + pointDistance(points[i], points[i - 1]))
    
    for i in range(1, last-first+1):
    #for (let i = 1, m = last - first i <= m i++) :
        u[i] /= u[last-first]
    
    return u

# Use Newton-Raphson iteration to find better root.
def findRoot (curve, point, u) :
    curve1 = []
    curve2 = []
    # Generate control vertices for Q'
    for i in range(3) : #FLAG
        curve1.append(pointMultiplyScalar(pointSubtract(curve[i + 1], curve[i]), 3))
    
    # Generate control vertices for Q''
    for i in range(2):
    #for (let i = 0 i <= 1 i++) :
        curve2.append(pointMultiplyScalar(pointSubtract(curve1[i + 1], curve1[i]), 2))
    
    # Compute Q(u), Q'(u) and Q''(u)
    pt = evaluate(3, curve, u)
    pt1 = evaluate(2, curve1, u)
    pt2 = evaluate(1, curve2, u)
    diff = pointSubtract(pt, point)
    df = pointDot(pt1, pt1) + pointDot(diff, pt2)
    # u = u - f(u) / f'(u)
    #return isMachineZero(df) ? u : u - pointDot(diff, pt1) / df
    return (u if isMachineZero(df) else (u-pointDot(diff,pt1)/df))

# Evaluate a bezier curve at a particular parameter value
def evaluate (degree, curve, t) :
    # Copy array
    tmp = curve[:]#.slice() FLAG
    # Triangle computation
    for i in range(1, degree+1):
    #for (let i = 1 i <= degree i++) :
        for j in range(degree):
        #for (let j = 0 j <= degree - i j++) :
            tmp[j] = pointAdd(pointMultiplyScalar(tmp[j], 1 - t), pointMultiplyScalar(tmp[j + 1], t))

    return tmp[0]

def addCurve (segments, curve)  :
    prev = segments[len(segments) - 1]
    prev.o = pointSubtract(curve[1], curve[0])
    segments.append(createSegment(curve[3], pointSubtract(curve[2], curve[3]), None))

# Use least-squares method to find Bezier control points for region.
def generateBezier (points, first, last, uPrime, tan1, tan2)  :
    epsilon = EPSILON
    #abs = Math.abs
    pt1 = points[first]
    pt2 = points[last]
    # Create the C and X matrices
    C = [[0, 0],[0, 0]]
    X = [0, 0]
    for i in range(0, last-first+1):
    #for (let i = 0, l = last - first + 1 i < l i++) :
        u = uPrime[i] 
        t = 1 - u
        b = 3 * u * t
        b0 = t * t * t
        b1 = b * t
        b2 = b * u
        b3 = u * u * u
        a1 = pointNormalize(tan1, b1)
        a2 = pointNormalize(tan2, b2)
        tmp = pointSubtract(pointSubtract(points[first + i], pointMultiplyScalar(pt1, b0 + b1)), pointMultiplyScalar(pt2, b2 + b3))
        C[0][0] += pointDot(a1, a1)
        C[0][1] += pointDot(a1, a2)
        # C[1][0] += a1.dot(a2)
        C[1][0] = C[0][1]
        C[1][1] += pointDot(a2, a2)
        X[0] += pointDot(a1, tmp)
        X[1] += pointDot(a2, tmp)
    
    # Compute the determinants of C and X
    detC0C1 = C[0][0] * C[1][1] - C[1][0] * C[0][1]
    alpha1=None
    alpha2=None
    if (abs(detC0C1) > epsilon) :
        # Kramer's rule
        detC0X = C[0][0] * X[1] - C[1][0] * X[0]
        detXC1 = X[0] * C[1][1] - X[1] * C[0][1]
        # Derive alpha values
        alpha1 = detXC1 / detC0C1
        alpha2 = detC0X / detC0C1
    
    else :
        # Matrix is under-determined, try assuming alpha1 == alpha2
        c0 = C[0][0] + C[0][1]
        c1 = C[1][0] + C[1][1]
        #alpha1 = alpha2 = abs(c0) > epsilon ? X[0] / c0 : abs(c1) > epsilon ? X[1] / c1 : 0
        alpha1 = alpha2 = X[0] / c0 if abs(c0) > epsilon else X[1] / c1 if abs(c1) > epsilon else 0
    
    # If alpha negative, use the Wu/Barsky heuristic (see text)
    # (if alpha is 0, you get coincident control points that lead to
    # divide by zero in any subsequent NewtonRaphsonRootFind() call.
    segLength = pointDistance(pt2, pt1)
    eps = epsilon * segLength
    handle1 = None
    handle2 = None
    if (alpha1 < eps or alpha2 < eps) :
        # fall back on standard (probably inaccurate) formula,
        # and subdivide further if needed.
        alpha1 = alpha2 = segLength / 3
    
    else :
        # Check if the found control points are in the right order when
        # projected onto the line through pt1 and pt2.
        line = pointSubtract(pt2, pt1)
        # Control points 1 and 2 are positioned an alpha distance out
        # on the tangent vectors, left and right, respectively
        handle1 = pointNormalize(tan1, alpha1)
        handle2 = pointNormalize(tan2, alpha2)
        if (pointDot(handle1, line) - pointDot(handle2, line) > segLength * segLength) :
            # Fall back to the Wu/Barsky heuristic above.
            alpha1 = alpha2 = segLength / 3
            handle1 = handle2 = None # Force recalculation
        
    
    # First and last control points of the Bezier curve are
    # positioned exactly at the first and last data points
    return [pt1, pointAdd(pt1, handle1 or pointNormalize(tan1, alpha1)), pointAdd(pt2, handle2 or pointNormalize(tan2, alpha2)), pt2]

# Given set of points and their parameterization, try to find
# a better parameterization.
def reparameterize (points, first, last, u, curve) :
    #for (let i = first i <= last i++) :
    for i in range(first, last+1):
        u[i - first] = findRoot(curve, points[i], u[i - first])
    
    # Detect if the new parameterization has reordered the points.
    # In that case, we would fit the points of the path in the wrong order.
    #for (let i = 1, l = u.length i < l i++) :
    for i in range (1, len(u)):
        if (u[i] <= u[i - 1]):
            return False
    
    return True


class error:
  def __init__(self, error, index):
    self.error = error
    self.index = index
# Find the maximum squared distance of digitized points to fitted curve.
def findMaxError (points, first, last, curve, u)  :
    index = math.floor((last - first + 1) / 2)
    maxDist = 0
    #for (let i = first + 1 i < last i++) :
    for i in range (first+1, last):
        P = evaluate(3, curve, u[i - first])
        v = pointSubtract(P, points[i])
        dist = v.x * v.x + v.y * v.y # squared
        if (dist >= maxDist) :
            maxDist = dist
            index = i
        
    return error(maxDist, index)

def fit (points, closed, error) :
    length = len(points)
    if length == 0 and type(length) == int:
        return []
    segments = [createSegment(points[0], None, None)]
    if length == 1:
      return segments
    fitCubic(points, segments, error, 0, length - 1, 
    pointSubtract(points[1], points[0]), 
    pointSubtract(points[length - 2], points[length - 1]))
    if (closed) :
        segments.pop(0)
        segments.pop()
    
    return segments

def fitCubic (points, segments, error, first, last, tan1, tan2):
    #  Use heuristic if region only has two points in it
    if (last - first == 1) :
        pt1 = points[first]
        pt2 = points[last]
        dist = pointDistance(pt1, pt2) / 3
        addCurve(segments, [pt1, pointAdd(pt1, pointNormalize(tan1, dist)), pointAdd(pt2, pointNormalize(tan2, dist)), pt2])
        return
    
    # Parameterize points, and attempt to fit curve
    uPrime = chordLengthParameterize(points, first, last)
    maxError = max(error, error * error)
    split=None
    parametersInOrder = True
    # Try not 4 but 5 iterations
    #for (let i = 0 i <= 4 i++) :
    for i in range(5):
        curve = generateBezier(points, first, last, uPrime, tan1, tan2)
        #  Find max deviation of points to fitted curve
        Max = findMaxError(points, first, last, curve, uPrime)
        if (Max.error < error and parametersInOrder) :
            addCurve(segments, curve)
            return
        
        split = Max.index
        # If error not too large, try reparameterization and iteration
        if (Max.error >= maxError):
            break
        parametersInOrder = reparameterize(points, first, last, uPrime, curve)
        maxError = Max.error
    
    # Fitting failed -- split at max error point and fit recursively
    tanCenter = pointSubtract(points[split - 1], points[split + 1])
    fitCubic(points, segments, error, first, split, tan1, tanCenter)
    fitCubic(points, segments, error, split, last, pointNegate(tanCenter), tan2)


def filterRepeats_Normalize(points, start_x, min_y, max_y, precision_b=250, first_offstroke_dist=30):
  filtered_points=[]
  for i in range(len(points)):
    if i==len(points)-1 or points[i][0] != points[i+1][0] or points[i][1] != points[i+1][1]:
      y_range=(max_y-min_y if max_y-min_y != 0 else 1)
      filtered_points.append((np.array(points[i])-np.array([start_x-first_offstroke_dist*y_range/precision_b, min_y]))*precision_b/y_range)
      #filtered_points.append((np.array(points[i])-np.array([min_x, min_y]))*precision_b/(max_y-min_y if max_y-min_y != 0 else 1))
  return filtered_points


# Assign parameter values to digitized points
# using relative distances between points.

def svgPath (segments, closed, precision) :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    if length==1:
      return 'M' + str(segments[0].p.x) + "," + str(segments[0].p.y) 
    
    if (not length):
        return ''
    for i in range(length):
        segment=segments[i]
        #repeat=False
        curX = segment.p.x
        curY = segment.p.y
        if (first) :
            parts.append('M' + formatPair(curX, curY))
        else :
            inX = curX + (segment.i.x if segment.i else 0)
            inY = curY + (segment.i.y if segment.i else 0)
            if (not (inX == curX and inY == curY and outX == prevX and outY == prevY)) :
                parts.append('c' +
                      formatPair(outX - prevX, outY - prevY) +
                      ' ' +
                      formatPair(inX - prevX, inY - prevY) +
                      ' ' +
                      formatPair(curX - prevX, curY - prevY))

        prevX = curX
        prevY = curY
        outX = curX + (segment.o.x if segment.o else 0)
        outY = curY + (segment.o.y if segment.o else 0)
        first = False

    return ''.join(parts)

def getSegmentsPathData (segments, closed, precision)  :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    if length==1:
      return ([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1]],(segments[0].p.x,segments[0].p.y),(segments[0].p.x,segments[0].p.y))
    
    if (not length):
        return []
    allSegmentData=[]
    for i in range(length):
      segmentData = []
      segment=segments[i]
      curX = segment.p.x
      curY = segment.p.y
      if (first) :
        startPoints=(curX,curY)
      else :
          inX = curX + (segment.i.x if segment.i else 0)
          inY = curY + (segment.i.y if segment.i else 0)
          if (not (inX == curX and inY == curY and outX == prevX and outY == prevY)) :
                # c = relative curveto:
              segmentData.extend([prevX, prevY, outX, outY, inX, inY, curX, curY])
              parts.append('c' +
                    formatPair(outX - prevX, outY - prevY) +
                    ' ' +
                    formatPair(inX - prevX, inY - prevY) +
                    ' ' +
                    formatPair(curX - prevX, curY - prevY))
              features = [segmentData[6]-segmentData[0], 
                          segmentData[7]-segmentData[1], 
                          math.sqrt((segmentData[0]-segmentData[2])**2+(segmentData[1]-segmentData[3])**2), 
                          math.sqrt((segmentData[6]-segmentData[4])**2+(segmentData[7]-segmentData[5])**2), 
                          math.atan2(segmentData[3]-segmentData[1], segmentData[2]-segmentData[0])-math.atan2(segmentData[7]-segmentData[1], segmentData[6]-segmentData[0]),
                          math.atan2(segmentData[5]-segmentData[7], segmentData[4]-segmentData[6])-math.atan2(segmentData[1]-segmentData[7], segmentData[0]-segmentData[6]), 
                          1]
              allSegmentData.append(features)
              endPoints=(segmentData[6],segmentData[7])

      prevX = curX
      prevY = curY
      outX = curX + (segment.o.x if segment.o else 0)
      outY = curY + (segment.o.y if segment.o else 0)
      first = False
    return (allSegmentData, startPoints, endPoints)

class options:
  def __init__(self, closed, tolerance, precision):
    self.closed = closed
    self.tolerance = tolerance
    self.precision=precision

def f1(p):
  return point(p.x, p.y)

def f2(p):
  return point(float(p[0]), float(p[1]))

def getSvgPath (points, options) :
    if (len(points) == 0 and type(len(points))==int) :
        return ''
    return svgPath(fit(list(map(f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)

def getFeatures (points, options)  :
    if (len(points) == 0 and type(len(points))==int) :
        return []
    return getSegmentsPathData(fit(list(map(f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)

In [41]:
error_files=[]
bezier_curve_errors=[]
parse_traces_errors=[]

In [42]:
from timeit import default_timer
def _parse_one_trace(trace_string):
  trace_split=trace_string.string
  if trace_split[0]=="\n":
    trace_split=trace_split[1:]
  if trace_split[-1]=="\n":
    trace_split=trace_split[:-1]
  trace_split=trace_split.split(",")
  a = [[float(s) for s in entry.strip().split(" ")[:2]] for entry in trace_split]
  return a


def getSvg(file, tolerance=15, precision_a=5, precision_b=250, first_offstroke_dist=30):
  #_, order = convert_relations(convert_inkml_to_lg(file))
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  a=""
  list_points=[]
  #order_dict=reorder(file.replace("INKMLs", "LGs").replace(".inkml", ".lg"))
  #for i in order:
  #  trace_string=trace_strings[i]
  #  list_points.append(_parse_one_trace(trace_string))
  for trace_string in trace_strings:
    list_points.append(_parse_one_trace(trace_string))
  array_points=tf.ragged.constant(list_points)
  mins=tf.math.reduce_min(array_points, axis=(0,1))
  maxs=tf.math.reduce_max(array_points, axis=(0,1))
  #min_x=float(mins[0])
  min_y=float(mins[1])
  max_y=float(maxs[1])
  for points in list_points:
    points=filterRepeats_Normalize(points,list_points[0][0][0],min_y,max_y,precision_b, first_offstroke_dist)
    path=getSvgPath(points, options(False,tolerance,precision_a))
    a+=path
  return a 

def parse_traces(file, order, tolerance=15, precision_a=5, precision_b=250, first_offstroke_dist=100):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  a=[]
  first = True
  prevEnd = None
  list_points=[]
  for i in order:
    try:
      trace_string=trace_strings[i]
    except:
      parse_traces_errors.append(trace_string)
    list_points.append(_parse_one_trace(trace_string))
  mins=tf.math.reduce_min(tf.ragged.constant(list_points), axis=(0,1))
  maxs=tf.math.reduce_max(tf.ragged.constant(list_points), axis=(0,1))
  min_y=float(mins[1])
  max_y=float(maxs[1])
  for points in list_points:
    points=filterRepeats_Normalize(points,list_points[0][0][0],min_y,max_y,precision_b, first_offstroke_dist)
    features=getFeatures(points, options(False,tolerance,precision_a))
    if first:
      a += [[first_offstroke_dist, precision_b/2-features[1][1], 0.0, 0.0, 0.0, 0.0, 0]]
      #NOTE if using the math.sqrt(dx**2....) then modify the above line as well
    else:
      try:
        dx=features[1][0]-prevEnd[0]
        dy=features[1][1]-prevEnd[1]
        #dx=features[1][len(features[1])-1][0] - prev[1][len(prev[1])-1][0]
        #dy=features[1][len(features[1])-1][1]-prev[1][len(prev[1])-1][1]
        #NOTE change indices 2 and 3 to 0 experimentally.
        #NOTE added extra on-strokes so the model can output blanks
        #NOTE changed from on-strokes to off-strokes
        a += [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]]
        a += [[dx, dy, 0.0, 0.0, 0.0, 0.0, 0]]
        a += [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]]
        #a += [[dx, dy, math.sqrt(dx**2/9+dy**2/9), math.sqrt(dx**2/9+dy**2/9), 0.0, 0.0, 0]]
      except:
        error_files.append(file)
      
    a += features[0]
    prevEnd = features[2]
    first=False
  a += [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]]
  a += [[first_offstroke_dist, precision_b/2-prevEnd[1], 0.0, 0.0, 0.0, 0.0, 0]]
  a += [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]]
  return a 

In [43]:
def convert_inkml_to_lg(file):
  return("FeatureData/LGs/"+file.split("/")[2]+"/"+file.split("/")[3][:-5]+"lg")
  #return ("/".join(file.split("/")[:6])+"/LGs/"+"/".join(file.split("/")[7:])[:-5]+"lg")

In [44]:
#old y data generation

from collections import defaultdict
# todo: Add something to return the sequence of unique identifiers ('x_1' etc.)
def parse_symbols_and_alignments(file):
  with open(file, "r") as f:
    string = f.read()
  #lg=[]
  rel=0 #BOOLEAN sorta
  objects=[]
  rels=defaultdict(lambda: "NoRel")
  for part in string.split("\n"):
    line=part.split(", ")
    if (line[0]=="# Relations from SRT:"):
      rel=1
    else:
      if (line[0]!=''):
        if rel:
          #if rel > 1:
          rels[(line[1], line[2])]=line[3]
          #rel+=1
          #prev=line[1]
        else:
          if (line[0][0]=="#"):
            continue
          del line[3]
          objects.append((line[1:3], [int(l) for l in line[3:]]))
  def key(obj):
    return min(obj[1])
  objects=sorted(objects, key=key)
  output=[]

  output.append(objects[0][0][1])
  prev=objects[0][0][0]
  #print(rels)
  for ob in objects[1:]:
    #print((ob[0][0], prev))
    output.append(rels[(prev, ob[0][0])])
    output.append(ob[0][1])
    prev=ob[0][0]
    #output.append(ob[0][1])
    #output.append(rels[ob[0][0]])
  '''try:
    output.append(objects[-1][0][1])
  except:
    print(file)'''
  #return(objects, rels)
  return(output)



In [45]:
class Node:
  def __init__(self, symbol, relations, parent, strokes):
    #self.id = id
    self.symbol = symbol
    self.relations = relations
    self.parent = parent
    self.strokes = strokes
  '''def get_root(self, nodes):
    node=self
    while(nodes[node].parent!=None):
      node=nodes[node].parent
    #node=Node(self.symbol,self.relations,self.parent,self.strokes)
    #while(nodes[node.parent].parent != None):
    #  node=nodes[node.parent]
    return node'''

def get_root(node, nodes):
    while(nodes[node].parent!=None):
      node=nodes[node].parent
    return node


#nodes={"A_1" : Node("A_1", "A", ["Right"], ["x_1"], None, ), }

In [46]:
from collections import defaultdict
failures=0
def lg_to_tree(file):
  with open(file, "r") as f:
    string = f.read()
  rel=0 #BOOLEAN sorta
  nodes=defaultdict(lambda: Node(None, [], None, []))
  nodes_inside_relation=defaultdict(list)
  for part in string.split("\n"):
    line=part.split(", ")
    if (line[0]=="# Relations from SRT:"):
      rel=1
    else:
      if (line[0]!=''):
        if rel:
          #if rel > 1:
          if line[3] == "Inside":
            nodes_inside_relation[line[1]].append(line[2])
          nodes[line[1]].relations.append([line[2],line[3]])
          nodes[line[2]].parent=line[1]
          #rels[(line[1], line[2])]=line[3]
          #rel+=1
          #prev=line[1]
        else:
          if (line[0][0]=="#"):
            continue
          del line[3]
          nodes[line[1]].symbol=line[2]
          nodes[line[1]].strokes=[int(stroke) for stroke in line[3:]]
  for node in nodes_inside_relation:
    targets=nodes_inside_relation[node]
    if len(targets)>2:
      raise ValueError("more than 2 inside relation detected")
    if len(targets)==2:
      if targets[1] in [r[0] for r in nodes[targets[0]].relations]:
        for i, rel in enumerate(nodes[node].relations):
          if rel[0]==targets[1]:
            target_index=i
            break
        del nodes[node].relations[target_index]
      elif targets[0] in [r[0] for r in nodes[targets[1]].relations]:
        for i, rel in enumerate(nodes[node].relations):
          if rel[0]==targets[0]:
            target_index=i
            break
        del nodes[node].relations[target_index]
      else:
        print("-----FAILURE------")
        failures+=1
        return None

  
  return(nodes)



In [47]:
#length=0


def traverse_tree(tree, current, order, length):
  strokes=current.strokes
  relations_order={"Above":1, "Below":2, "Inside":3, "Sup":4, "Sub":5, "Right":6}
  relations=sorted(current.relations, key = lambda rel: relations_order[rel[1]])
  def _add_current_strokes(strokes,length):
    strokes=sorted(strokes)
    for x in range(len(strokes)):
      order[strokes[x]]=length
      length+=1
    return length
  def _traverse_children(tree,relations,order,length):
    for i in range(len(relations)):
      order_length=traverse_tree(tree, tree[relations[i][0]], order, length)
      order=order_length[0]
      length=order_length[1]
    return length
  if len(relations)>0:
    if relations[0][1]=="Above":
      length = _traverse_children(tree,[relations[0]],order,length)
      length = _add_current_strokes(strokes,length)
      length = _traverse_children(tree,[relations[1]],order,length)
      #print("ABOVE",current.symbol, length)
      #length = _add_current_strokes(strokes,length)
    else:
      length = _add_current_strokes(strokes,length)
      #print("OTHER",current.symbol,length)
      length = _traverse_children(tree,relations,order,length)
  else:
    length = _add_current_strokes(strokes,length)
  #global length
  
  
  #if len(relations)==0:
  #print(current.symbol, length)
  return((order, length))

def reorder(file):
  tree=lg_to_tree(file)
  order = []
  root = get_root(list(tree.keys())[0], tree)
  #for i in range(len(root.strokes)):
    #order[root.strokes[i]]=i
  #global length
  length=0
  return(traverse_tree(tree, tree[root], order, length)[0])

In [48]:
def convert_sequence(sequence):
    symbols=sequence[::2]
    relations=sequence[1::2]
    return(symbols, relations)
#NOTe MAKE UTF NOT APPEAR FOR SUMS

In [49]:
#\frac{9x^2 + 5}{2}

#symbols = ['9','x','2','-','2','fractionbar','5','\\cdot ','x','2','fractionbar','5']
#relations = ["Right", "Sup", "DFS", "UTF", "NTB", "BTD", "UFD", "Right", "Sub", "OST", "BTD"]

symbols, relations=convert_sequence(['1', 'NTB', '-', 'BTD', 'r', 'Sup', '2', 'DFS-UFD', '=', 'UTF', '1', 'NTB', '-', 'BTD', '(', 'Right', 'R', 'Right', '-', 'Right', 'm', 'Right', ')', 'Sup', '2', 'DFS-UFD', '+', 'UTF', '1', 'NTB', '-', 'BTD', '(', 'Right', 'R', 'Right', '+', 'Right', 'm', 'Right', ')', 'Sup', '2'])

def parse_latex(symbols, relations):
  utf=0
  open_brackets=0
  latex_string=symbols[0]
  for i in range(len(relations)):
    s=symbols[i+1]
    full_rel=relations[i]
    split_rel=full_rel.split('-')
    add_symbol=True
    for rel in split_rel:
      if rel in ["DFS", "UFS", "OFI", "UFD", "UFL", "DFL"]:
        latex_string+="}"
        open_brackets-=1
      if rel == "NTB":
        add_symbol=False
        if utf:
          latex_string+=("}{")
          utf-=1
        else:
          latex_string="\\frac{"+latex_string+"}{"
          open_brackets+=1
      if rel == "UTF":
        latex_string+=("\\frac{"+s)
        open_brackets+=1
        utf+=1
        add_symbol=False
      if rel in ["Sub", "LB", "DTI"]:
        latex_string+=("_{")
        open_brackets+=1
      if rel == "Sup":
        latex_string+=("^{")
        open_brackets+=1
      if rel == "Inside":
        latex_string+=("{")
        open_brackets+=1
      if rel == "STS":
        latex_string+=("}_{")
      if rel == "ITL":
        latex_string+=("}^{")
      if rel == "Radical":
        latex_string+="["
      if rel == "RTI":
        latex_string+="]{"
        open_brackets+=1
    if add_symbol:
      latex_string+=s
  latex_string+=("}")*open_brackets
  return latex_string
  
parse_latex(symbols, relations)

'\\frac{1}{r^{2}}=\\frac{1}{(R-m)^{2}}+\\frac{1}{(R+m)^{2}}'

In [50]:
missing_relations=set()
missing_relations_freq=defaultdict(lambda: 0)
pop_errors=[]

In [51]:
arr=[]

In [52]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [53]:
import pickle as pkl
from collections import defaultdict
with open("FeatureData/Vocab.pkl", "rb") as f:
    vocab = pkl.load(f)
last_symbol=None
new_relations=[]
new_rels=defaultdict(lambda: [])
def traverse_relations(tree, current, new_rels, incoming_rel, prev_symbol, order):
  if tree[current].symbol=="\sum":
    relations_order={"Below":1, "Above":2, "Inside":3, "Sup":4, "Sub":5, "Right":6}
  else:
    relations_order={"Above":1, "Below":2, "Inside":3, "Sup":4, "Sub":5, "Right":6}
  
  #print(current)
  relations=sorted(tree[current].relations, key = lambda rel: relations_order[rel[1]])
  last_symbol=current
  if len(relations) > 0:
    no_rel=False
    #tree[current].symbol == "-" and 
    if relations[0][1]=="Above":
      if tree[current].symbol == "\int":
        relations[0][1]="Sup"
        if relations[1][1]=="Below":
          relations[1][1]="Sub"
        new_rels[current].append("Sup")
      elif tree[current].symbol == "\sum":
        new_rels[current].append("STL")
      elif tree[current].symbol == "\sqrt":
        new_rels[current].append("Radical")
      elif prev_symbol and len(new_rels[prev_symbol]):
        #if tree[prev_symbol].relations
        no_rel=True
        if new_rels[prev_symbol][-1]=="Right":
          new_rels[prev_symbol].pop()
        #print("prev",prev_symbol)
        new_rels[prev_symbol].append("UTF")
      else:
        no_rel=True
    elif relations[0][1]=="Below":
      if tree[current].symbol == "\sum":
        new_rels[current].append("DTI")
      else:
        new_rels[current].append("LB")
    else:
      new_rels[current].append(relations[0][1])
    if not no_rel and len(new_rels[current])==1:
      strokes=sorted(tree[current].strokes)
      #print(strokes)
      for x in strokes:
        order.append(x)
    for i in range(len(relations)):
      thing=traverse_relations(tree, relations[i][0], new_rels, relations[i][1], last_symbol, order)#CHANGED
      last_symbol=thing[0]
      new_rels=thing[1]
      #length=thing[2]
      order=thing[2]
      if relations[i][1]=="Above":
        if tree[current].symbol=="-":
          tree[current].symbol="frac"
          new_rels[current].append("BTD")
          if len(new_rels[current])==1:
            strokes=sorted(tree[current].strokes)
            for x in strokes:
              order.append(x)
              #length+=1
  else:
    new_rels[current]=[]
  add=True
  if incoming_rel=="Above":
    if tree[prev_symbol].symbol=="-" or tree[prev_symbol].symbol=="frac": #CHANGE BECAUSE PREV SYMBOL IS THE LEAF OF SUM NOT SUM ITSELF
      #do stuff with sum
      new_rels[last_symbol].append("NTB")
    elif tree[prev_symbol].symbol=="\sqrt":
      new_rels[last_symbol].append("RTI")
    else:
      new_rels[last_symbol].append("DFL")
  elif incoming_rel=="Below":
    if tree[prev_symbol].symbol=="\lim":
      new_rels[last_symbol].append("UFL")
    elif tree[prev_symbol].symbol=="\sum":
      new_rels[last_symbol].append("ITL")
    else:
      new_rels[last_symbol].append("UFD")
  elif incoming_rel=="Sub":
    new_rels[last_symbol].append("UFS")
  elif incoming_rel=="Sup":
    if tree[prev_symbol].symbol=="\int":
      new_rels[last_symbol].append("STS")
    else:
      new_rels[last_symbol].append("DFS")
  elif incoming_rel=="Inside":
    new_rels[last_symbol].append("OFI")
  else:
    add=False
  if add:
    if len(new_rels[last_symbol])==1:
      strokes=sorted(tree[last_symbol].strokes)
      for x in strokes:
        order.append(x)
  return (last_symbol, new_rels, order)

def convert_relations(file):
  tree=lg_to_tree(file)
  root = get_root(list(tree.keys())[0], tree)
  order=[]
  new_rels=defaultdict(lambda: [])
  new_rels["S"]=["Right"]
  try:
    traverse_output=traverse_relations(tree, root, new_rels, "Right", "S", order)
    new_rels=traverse_output[1]
    order=traverse_output[2]
    output=[]
    not_in_vocab=False
    for symbol in new_rels:
      if symbol != "S":
        output.append(vocab[tree[symbol].symbol])
      if len(new_rels[symbol]):
        relation="-".join(new_rels[symbol])
        if relation in vocab:
          output.append(vocab[relation])
        else:
          missing_relations.add(relation)
          missing_relations_freq[relation]+=1
          not_in_vocab=True
          relation=relation.split("-")
          if relation.count("Right")>1 or relation.count("Inside")>1 or relation.count("Sup")>1 or relation.count("Sub")>1 or relation.count("BTD")>1:
            return (0, None) #0 is used to indentify this specific error from this function's output when generating the data
  except:
    print("pop error: "+file)
    pop_errors.append(file)
    return (None, None)

  if not_in_vocab:
    return (None, None)
  if len(output)%2==0:
    output.append(0)
  #if len(output)%2==0:
  #  output.pop()
  return (output, order)

def get_latex(file):
  arr=[]
  tree=lg_to_tree(file)
  root = get_root(list(tree.keys())[0], tree)
  order=[]
  new_rels=defaultdict(lambda: [])
  new_rels=traverse_relations(tree, root, new_rels, None, None, order)[1]
  not_in_vocab=False
  for symbol in new_rels:
    if tree[symbol].symbol=="COMMA":
      arr.append(",")
    else:
      arr.append(tree[symbol].symbol)
    if len(new_rels[symbol]):
      relation="-".join(new_rels[symbol])
      if relation=="DFS-Sub":
        relation="STS"
      if relation in vocab:
        arr.append(relation)
      else:
        print("missing: " +relation)
        missing_relations.add(relation)
        missing_relations_freq[relation]+=1
        not_in_vocab=True
  
  if not_in_vocab:
    return (None, None)
  
  if len(arr)%2==0:
    arr.pop()
  converted_sequence=convert_sequence(arr)
  return parse_latex(converted_sequence[0], converted_sequence[1])


In [54]:
def add_to_vocab(symbol, vocab, index):
    for key in vocab:
        if vocab[key]>=index:
            vocab[key]+=1
    vocab[symbol]=index
    vocab=dict(sorted(vocab.items(), key=lambda item: item[1]))
    #with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "wb") as f:
    #    pkl.dump(vocab, f)
def remove_from_vocab(vocab, index, symbol=None):
    for key in vocab:
        if vocab[key]>=index:
            vocab[key]-=1
    if symbol:
        vocab[symbol]=index
    vocab=dict(sorted(vocab.items(), key=lambda item: item[1]))
    #with open("C:/Users/aiden/Downloads/ProjectData/Vocab.pkl", "wb") as f:
    #    pkl.dump(vocab, f)

In [55]:
#add_to_vocab("frac", vocab, 48)
#vocab=dict(sorted(vocab.items(), key=lambda item: item[1]))
#with open("FeatureData/Vocab.pkl", "wb") as f:
#    pkl.dump(vocab, f)
#with open("FeatureData/index_to_symbol.pkl", "wb") as f:
#    pkl.dump(index_to_symbol, f)

In [ ]:
import pickle as pkl
import os
years=["2012", "2013", "2014train", "2014test", "2016", "2019"]
#years = ["2019"]
#train=False
#name="LG"
error_files=[]
bezier_curve_errors=[]
parse_traces_errors=[]
pop_errors=[]
bad_relation_files=[]
missing_relations=set()
missing_relations_freq=defaultdict(lambda: 0)
failures=0
#datapath="C:/Users/aiden/Downloads/ProjectData/"+year+"/"+ ("Train/" if train else "Test/") + name
X=[]
Y=[]
count=0
with open("FeatureData/Vocab.pkl", "rb") as f:
    vocab = pkl.load(f)
for year in years:
    directory="FeatureData/INKMLs/"+year+"/"
    for filename in os.listdir(directory):
        #the filename unincluding .inkml
        lgpath=convert_inkml_to_lg(directory+filename)
        #lgpath="C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/"+year+"/"+filename[:len(filename)-6]+".lg"
        if os.path.exists(lgpath):
            LG, order = convert_relations(lgpath)
            if order and LG:
                X.append(parse_traces(directory+filename, order, tolerance=20))
                Y.append(LG)
            else:
                if LG==0:
                    bad_relation_files.append(lgpath)
        else:
            print("missing file:")
            print(lgpath)
            print(filename)
        count += 1

        if count % 100 == 0: print(count)
datapath="FeatureData/"
with open(datapath+"X_Y_2", "wb") as f:
    pkl.dump((X,Y), f)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
missing file:
FeatureData/LGs/2014train/formulaire021-equation023 (8bf4d083).lg
formulaire021-equation023 (8bf4d083).inkml
1700
1800
1900
2000
2100
2200
missing file:
FeatureData/LGs/2014train/formulaire019-equation025 (c9228215).lg
formulaire019-equation025 (c9228215).inkml
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
missing file:
FeatureData/LGs/2014train/formulaire019-equation029 (1acd9714).lg
formulaire019-equation029 (1acd9714).inkml
5100
5200
5300
5400
missing file:
FeatureData/LGs/2014train/formulaire019-equation030 (6987c839).lg
formulaire019-equation030 (6987c839).inkml
5500
5600
missing file:
FeatureData/LGs/2014train/formulaire019-equation028 (87b51f17).lg
formulaire019-equation028 (87b51f17).inkml
5700
5800
5900
6000
6100
missing file:
FeatureData/LGs/2014train/formulaire019-equation026 (7c37a40d).lg
formulair

In [ ]:
#index_to_symbol = {}
#for key in vocab.keys():
#    index_to_symbol[vocab[key]] = key

In [ ]:
getSvg("FeatureData/INKMLs/2014train/formulaire020-equation005.inkml")

In [ ]:
#print(missing_relations)
print(dict(sorted(missing_relations_freq.items(), key=lambda item: item[1])))
#print(sorted(missing_relations_freq, key = lambda rel: missing_relations_freq[rel]))
'''print("pop_errors:")
print(pop_errors)
print("error_files")
print(error_files)
print("bezier_curve_errors")
print(bezier_curve_errors)
print("bad_relation_files:")
print(bad_relation_files)'''

In [ ]:
Y[3]

In [ ]:
print(len(error_files))
print(len(pop_errors))
print(len(bad_relation_files))
print(bezier_curve_errors)
print(sum([missing_relations_freq[rel] for rel in missing_relations_freq.keys()]))

In [ ]:
#testing parse_traces
file="C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2014train/109_leissi.inkml"
#print(convert_relations(file))
#convert_relations(file)
parse_traces(file, convert_relations(convert_inkml_to_lg(file))[1])

In [ ]:
vocab

In [ ]:
get_latex("C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/2014train/109_leissi.lg")

In [ ]:
with open("FeatureData/Vocab.pkl", "rb") as f:
    vocab=pkl.load(f)

In [ ]:
vocab

In [ ]:
datapath="FeatureData/"
with open(datapath+"X_Y", "rb") as f:
    X, Y = pkl.load(f)

In [ ]:
length = len(X)
vocab_size = len(vocab)
X_train = []
Y_train = []
X_test = []
Y_test = []
X_eval = []
Y_eval = []
for i in range(length):
    if (i%10>1):
        X_train.append(X[i])
        Y_train.append(Y[i])
    elif (i%10):
        X_eval.append(X[i])
        Y_eval.append(Y[i])
    else:
        X_test.append(X[i])
        Y_test.append(Y[i])


In [ ]:
getSvg("C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2012/001-equation000.inkml")

In [ ]:
datapath="FeatureData/"

In [ ]:
with open(datapath+"Train", "wb") as f:
    pkl.dump((X_train, Y_train), f)
with open(datapath+"Test", "wb") as f:
    pkl.dump((X_test, Y_test), f)
with open(datapath+"Eval", "wb") as f:
    pkl.dump((X_eval, Y_eval), f)

In [ ]:
with open(datapath+"Train", "rb") as f:
    X_train, Y_train = pkl.load(f)
with open(datapath+"Test", "rb") as f:
    X_test, Y_test = pkl.load(f)
with open(datapath+"Eval", "rb") as f:
    X_eval, Y_eval = pkl.load(f)

In [ ]:
print(vocab)

In [ ]:
NUM_FEATURES=7
len_train=len(X_train)
len_test=len(X_test)
max_X_train_len = max([len(sample) for sample in X_train])
max_Y_train_len = max([len(sample) for sample in Y_train])
max_X_test_len = max([len(sample) for sample in X_test])
max_Y_test_len = max([len(sample) for sample in Y_test])
padding_vector=np.ones(NUM_FEATURES)
eos_index=0

In [ ]:
X_train_data_len=[len(data) for data in X_train]
Y_train_data_len=[len(data) for data in Y_train]
X_test_data_len=[len(data) for data in X_test]
Y_test_data_len=[len(data) for data in Y_test]

In [ ]:
padded_X_train=np.zeros((len_train, max_X_train_len, NUM_FEATURES))+padding_vector
padded_Y_train=np.zeros((len_train, max_Y_train_len))+eos_index
padded_X_test=np.zeros((len_test, max_X_test_len, NUM_FEATURES))+padding_vector
padded_Y_test=np.zeros((len_test, max_Y_test_len))+eos_index

In [ ]:
for i in range(len_train):
    sample_X = np.array(X_train[i])
    padded_X_train[i,:len(sample_X),:]=sample_X
    sample_Y = np.array(Y_train[i])
    padded_Y_train[i,:len(sample_Y)]=sample_Y
        
for i in range(len_test):
    sample_X = np.array(X_test[i])
    padded_X_test[i,:len(sample_X),:]=sample_X
    sample_Y = np.array(Y_test[i])
    padded_Y_test[i,:len(sample_Y)]=sample_Y


In [ ]:
getSvg(inkml_file_path+"2012/001-equation000.inkml")

In [ ]:
vocab

In [ ]:
X_train[0]

In [ ]:
Y[0]

In [ ]:
print(len(Y_train))
print(len(X_test))
print(len(X_eval))